In [1]:
import json
import itertools
from collections import defaultdict
import numpy as np
import pandas as pd

# ------------ 1.  Load your data  -------------------------------------------
with open("train.json", "r", encoding="utf-8") as f:
    # If your file is a list: data = json.load(f)
    # If it is an object keyed by IDs (what you showed):  use .values()
    data = json.load(f)

# ------------ 2.  Discover all unique ingredients  --------------------------
ingredient_to_idx = {}
for recipe in data:
    for raw in recipe["ingredients"]:
        ing = raw.strip().lower()
        if ing not in ingredient_to_idx:
            ingredient_to_idx[ing] = len(ingredient_to_idx)

n = len(ingredient_to_idx)
matrix = np.zeros((n, n), dtype=int)

# ------------ 3.  Update co-occurrence counts  ------------------------------
for recipe in data:
    ing_set = {raw.strip().lower() for raw in recipe["ingredients"]}

    # Count every unordered pair once per recipe
    for a, b in itertools.combinations(ing_set, 2):
        i, j = ingredient_to_idx[a], ingredient_to_idx[b]
        matrix[i, j] += 1
        matrix[j, i] += 1

    # Optional: store recipe counts on the diagonal
    for ing in ing_set:
        idx = ingredient_to_idx[ing]
        matrix[idx, idx] += 1          # drop this loop if you want zeros on the diagonal



In [11]:
# delete uncommon ingredients: less than 1000 recipes
min_recipes = 15
diag = np.diag(matrix)
keep_indices = np.where(diag >= min_recipes)[0]

# Filter the matrix
filtered_matrix = matrix[np.ix_(keep_indices, keep_indices)]

# Update the ingredient mapping
idx_to_ingredient = {i: ing for ing, i in ingredient_to_idx.items()}
new_ingredient_list = [idx_to_ingredient[i] for i in keep_indices]
filtered_ingredient_to_idx = {ing: i for i, ing in enumerate(new_ingredient_list)}

print(filtered_matrix)
print(len(filtered_matrix))

[[7380 1308  399 ...    8    4    5]
 [1308 4438  246 ...    2    0    3]
 [ 399  246 1896 ...    0    0    1]
 ...
 [   8    2    0 ...   16    0    0]
 [   4    0    0 ...    0   15    0]
 [   5    3    1 ...    0    0   20]]
731


In [13]:
# if dataframe takes too much memory (it does):
from scipy import sparse
import numpy as np

# matrix is the NumPy array you built earlier
mat_csr = sparse.csr_matrix(matrix)          # compresses rows & drops zero
sparse.save_npz("cooc.npz", mat_csr, compressed=True)

#uncompress w/ mat_csr = sparse.load_npz("cooc.npz")

In [ ]:
# Show the first few rows
print(len(df))
df.to_csv("co_occurrence.csv")

6703
